In [1]:
# data handling
import numpy as np
import pandas as pd

# data visualizations
import seaborn as sns
import matplotlib.pyplot as plt

# feature scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# feature selection
from sklearn.feature_selection import RFE

# machine learning algorithms
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import naive_bayes 
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier
import xgboost as xgb

# dimensionality reduction with PCA
from sklearn.decomposition import PCA

# accuracy metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-storm-4/Data-dictionary.xlsx
/kaggle/input/data-storm-4/DataStorm4.0_KaggleProblemStatement.pdf
/kaggle/input/data-storm-4/Store-info.csv
/kaggle/input/data-storm-4/Testing-data.csv
/kaggle/input/data-storm-4/Historical-transaction-data.csv


In [3]:
store_db = pd.read_csv('/kaggle/input/data-storm-4/Store-info.csv')
transaction_db = pd.read_csv('/kaggle/input/data-storm-4/Historical-transaction-data.csv')
test_db = pd.read_csv('/kaggle/input/data-storm-4/Testing-data.csv')

## Creating the Master Table of Data based on Transactions and the corresponding Store 

In [4]:
table = pd.merge(transaction_db, store_db)

### Splitting data into smaller ones on the Master Data

In [5]:
# Amount of money spent in rupees for each transaction by each person on a particular item
table['amount'] = table['item_price'] * table['quantity_sold']

# Handling the date
table['transaction_date'] = table['transaction_date'].str.split("T",n=1,expand=True)[0]
table['transaction_date'] = pd.to_datetime(table['transaction_date'],format="%Y-%m-%d")

# Splitting the date components
table['year'] = table['transaction_date'].dt.year
table['month'] = table['transaction_date'].dt.month
table['date'] = table['transaction_date'].dt.day
table['weekday'] = table['transaction_date'].dt.weekday 

In [6]:
price_item_modes = table.groupby(['item_price'])['item_description'].agg(pd.Series.mode)

for i in price_item_modes.index:
    if type(price_item_modes[i]) == np.ndarray:
        price_item_modes[i] = np.NaN
        
table['item_description'] = table['item_description'].fillna(table['item_price'].map(price_item_modes))

In [7]:
table.dropna(subset=['item_description'], inplace=True)

In [8]:
# Obtaining the beverage type and the volume of the beverage in each transaction
item_desc_split = table['item_description'].str.rsplit(" ",n=1,expand=True)
table['beverage_description'] = item_desc_split[0]
table['volume_type'] = item_desc_split[1]

# Obtaining the volume of the beverage bottles as millilitres
# code obtained from https://stackoverflow.com/a/61041274 
vol_data = table.volume_type.str.extract(r'(?i)\b(\d+(?:\.\d+)?)\s*(ML|L)\b', expand=True)
vol_data[0] = vol_data[0].astype(float)

vol_data = vol_data.replace(['ML','L'],[0,1])
vol_data[1].astype(float)

vol_data[0] = vol_data[0] + vol_data[1]*999*vol_data[0]

table['volume_ml'] = vol_data[0]

In [9]:
reordered_cols = ['month','date','weekday','invoice_id','customer_id','beverage_description',
                  'volume_ml','item_price','quantity_sold','amount','shop_id','shop_area_sq_ft',
                  'shop_profile']
table = table[reordered_cols]
table

,month,date,weekday,invoice_id,customer_id,beverage_description,volume_ml,item_price,quantity_sold,amount,shop_id,shop_area_sq_ft,shop_profile
0,12,11,5,147.0,BGXA,ORANGE BARLEY,1500.0,220,2,440,SHOP008,678,Moderate
1,12,13,0,484.0,VN7V,TONIC PET,500.0,160,2,320,SHOP008,678,Moderate
2,12,13,0,484.0,VN7V,CREAM SODA,1000.0,150,2,300,SHOP008,678,Moderate
3,12,10,4,1000053.0,VT9C,GINGER BEER,1500.0,220,1,220,SHOP008,678,Moderate
4,12,10,4,1000057.0,8QLS,GINGER BEER,1500.0,440,1,440,SHOP008,678,Moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...
473969,12,9,3,7030409.0,QVA7,SODA,500.0,70,1,70,SHOP072,617,High
473970,12,11,5,7030809.0,HXMG,FIT O ORANGE,200.0,140,2,280,SHOP072,617,High
473971,12,11,5,7030820.0,OUH2,BOTTLED DRINKING WATER,500.0,35,1,35,SHOP072,617,High
473972,10,21,3,NaN,5IJM,GINGER BEER,1500.0,220,1,220,SHOP072,617,High


## Handling Missing Values

There are two features in the original dataset that have missing values. 
1. Invoice Number
2. Item Description (now split as beverage_description & volume_ml)

To impute the missing invoice numbers we will assume that all transactions that were done using the same customer_id on the same date to be considered as a single invoice. Since we only to identify whether these transactions belong to the same invoice, it is not necessary to identify the correct the invoice id number. Instead we will create a new invoice number starting from the last invoice recorded in the table (i.e. highest number in the invoice_id column)

In [10]:
no_inv_db = table.loc[table.invoice_id.isna()]
invoice_index = no_inv_db.sort_values(['customer_id','month','date']).groupby(
    ['customer_id','month','date']).count().index

invoice_i = table.invoice_id.max() + 1
for idx in invoice_index:
    no_inv_db.loc[(no_inv_db.customer_id == idx[0]) &
                  (no_inv_db.month == idx[1]) & 
                  (no_inv_db.date == idx[2]),'invoice_id'] = invoice_i
    invoice_i += 1
    
table.loc[table.invoice_id.isna()] = no_inv_db

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


It is idenitified that shop with id 'SHOP008' has only being functioning in the month December and hence will be dropped from the training data

table = table.loc[table.shop_id != 'SHOP008']
store_db = store_db.loc[store_db.shop_id != 'SHOP008']

In [11]:
table.loc[table.shop_id == 'SHOP047']

,month,date,weekday,invoice_id,customer_id,beverage_description,volume_ml,item_price,quantity_sold,amount,shop_id,shop_area_sq_ft,shop_profile
21177,10,17,6,30956.0,4OFQ,SODA,1000.0,110,1,110,SHOP047,528,Moderate
21178,10,20,2,31418.0,KQYP,ORANGE BARLEY,1500.0,220,1,220,SHOP047,528,Moderate
21179,11,1,0,33467.0,WCZT,GINGER BEER,1500.0,220,2,440,SHOP047,528,Moderate
21180,11,6,5,34265.0,KD8T,FIT O MIXED FRUIT,200.0,60,1,60,SHOP047,528,Moderate
21181,11,6,5,34265.0,KD8T,GINGER BEER,1500.0,220,2,440,SHOP047,528,Moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22864,11,18,3,2026696.0,ULVU,NECTO,500.0,100,1,100,SHOP047,528,Moderate
22865,11,28,6,2027937.0,5C5L,SODA,1000.0,110,2,220,SHOP047,528,Moderate
22866,10,23,5,3011507.0,IN28,LEMONADE,1500.0,220,2,440,SHOP047,528,Moderate
22867,10,23,5,3011514.0,4QJ5,GINGER BEER,400.0,600,7,4200,SHOP047,528,Moderate


## Forming Aggregate Data Regarding each shop

In [12]:
store_table = store_db.copy()

store_table = store_table.loc[store_table.shop_id != "SHOP008"]
store_table

In [13]:
def add_new_feature(database, col_name, new_series):
    database[col_name] = 0
    i=0
    for idx in database.shop_id:
        database.loc[i, col_name] = new_series.loc[idx][0]
        i+=1
    return database

In [14]:
all_shop_index = store_table.shop_id
all_shop_index = all_shop_index.dropna()
all_shop_index

0      SHOP047
1      SHOP009
2      SHOP083
3      SHOP117
4      SHOP042
        ...   
119    SHOP087
120    SHOP050
121    SHOP061
122    SHOP056
123    SHOP070
Name: shop_id, Length: 124, dtype: object

In [15]:
tot_sales = pd.DataFrame( table.groupby('shop_id', sort=False)['amount'].sum())
store_table = add_new_feature(store_table, 'tot_sales', tot_sales)

In [16]:
oct_sales = pd.DataFrame(table.loc[table.month == 10].groupby('shop_id',sort=False)['amount'].sum(),index=all_shop_index)
store_table = add_new_feature(store_table, 'oct_sales', oct_sales)

In [17]:
nov_1_sales = pd.DataFrame(table.loc[(table.month == 11) & (table.date <= 15)].groupby('shop_id',sort=False)['amount'].sum(),index=all_shop_index)
store_table = add_new_feature(store_table, 'nov_1_sales', nov_1_sales)

In [18]:
nov_2_sales = pd.DataFrame(table.loc[(table.month == 11) & (table.date > 15)].groupby('shop_id',sort=False)['amount'].sum(),index=all_shop_index)
store_table = add_new_feature(store_table, 'nov_2_sales', nov_2_sales)

In [19]:
dec_sales = pd.DataFrame(table.loc[table.month == 12].groupby('shop_id',sort=False)['amount'].sum(),index=all_shop_index)
store_table = add_new_feature(store_table, 'dec_sales', dec_sales)

In [20]:
tot_customers = pd.DataFrame( table.groupby('shop_id',sort=False)['customer_id'].nunique(), index=all_shop_index).astype(int)
store_table = add_new_feature(store_table, 'tot_customers', tot_customers)

In [21]:
oct_customers = pd.DataFrame(table.loc[table.month == 10].groupby('shop_id',sort=False)['customer_id'].nunique(),index=all_shop_index)
store_table = add_new_feature(store_table, 'oct_customers', oct_customers)

In [22]:
nov_1_customers = pd.DataFrame(table.loc[(table.month == 11) & (table.date <= 15)].groupby('shop_id',sort=False)['customer_id'].nunique(),index=all_shop_index)
store_table = add_new_feature(store_table, 'nov_1_customers', nov_1_customers)

In [23]:
nov_2_customers = pd.DataFrame(table.loc[(table.month == 11) & (table.date > 15)].groupby('shop_id',sort=False)['customer_id'].nunique(),index=all_shop_index)
store_table = add_new_feature(store_table, 'nov_2_customers', nov_2_customers)

In [24]:
dec_customers = pd.DataFrame(table.loc[table.month == 12].groupby('shop_id',sort=False)['customer_id'].nunique(),index=all_shop_index)
store_table = add_new_feature(store_table, 'dec_customers', dec_customers)

In [25]:
visit_per_customer = pd.DataFrame(table.groupby('shop_id')['customer_id'].value_counts())
repeat_customers = visit_per_customer.loc[visit_per_customer.customer_id > 1].groupby('shop_id').count()
store_table = add_new_feature(store_table, 'repeat_customers', repeat_customers)

In [26]:
frequent_customers = visit_per_customer.loc[visit_per_customer.customer_id > 12].groupby('shop_id').count()
frequent_customers = pd.DataFrame(frequent_customers, index=all_shop_index)
store_table = add_new_feature(store_table, 'frequent_customers', frequent_customers)

In [27]:
customers_180_bottles = pd.DataFrame(table.loc[table.volume_ml == 180].groupby('shop_id',sort=False)['customer_id'].nunique(),index=all_shop_index)
store_table = add_new_feature(store_table, 'customers_180_bottles', customers_180_bottles)

In [28]:
customers_200_bottles = pd.DataFrame(table.loc[table.volume_ml == 200].groupby('shop_id',sort=False)['customer_id'].nunique(),index=all_shop_index)
store_table = add_new_feature(store_table, 'customers_200_bottles', customers_200_bottles)

In [29]:
customers_400_bottles = pd.DataFrame(table.loc[table.volume_ml == 400].groupby('shop_id',sort=False)['customer_id'].nunique(),index=all_shop_index)
store_table = add_new_feature(store_table, 'customers_400_bottles', customers_400_bottles)

In [30]:
customers_500_bottles = pd.DataFrame(table.loc[table.volume_ml == 500].groupby('shop_id',sort=False)['customer_id'].nunique(),index=all_shop_index)
store_table = add_new_feature(store_table, 'customers_500_bottles', customers_500_bottles)

In [31]:
customers_1000_bottles = pd.DataFrame(table.loc[table.volume_ml == 1000].groupby('shop_id',sort=False)['customer_id'].nunique(),index=all_shop_index)
store_table = add_new_feature(store_table, 'customers_1000_bottles', customers_1000_bottles)

In [32]:
customers_1500_bottles = pd.DataFrame(table.loc[table.volume_ml == 1500].groupby('shop_id',sort=False)['customer_id'].nunique(),index=all_shop_index)
store_table = add_new_feature(store_table, 'customers_1500_bottles', customers_1500_bottles)

In [33]:
customers_1500_bottles.loc['SHOP047']

customer_id    422
Name: SHOP047, dtype: int64

In [34]:
table.beverage_description.unique()

array(['ORANGE BARLEY', 'TONIC PET', 'CREAM SODA', 'GINGER BEER',
       'CREAM SODA APPLE POP', 'SODA', 'BOTTLED DRINKING WATER',
       'LEMONADE', 'KIK COLA', 'STRAWBERRY MILK', 'SODA PET',
       'FIT O MIXED FRUIT', 'NECTO', 'FIT O ORANGE', 'FIT O MANGO',
       'GINGER BEER SUGAR FREE', 'ORANGE CRUSH', 'CHOCOLATE MILK',
       'TWISTEE APPLE', 'DRY GINGER ALE PET', 'LIME CRUSH JUICE'],
      dtype=object)

In [35]:
soft_drinks = ['ORANGE BARLEY','TONIC PET','CREAM SODA','GINGER BEER','CREAM SODA APPLE POP','SODA',
               'LEMONADE','KIK COLA','SODA PET','NECTO','GINGER BEER SUGAR FREE','ORANGE CRUSH',
               'DRY GINGER ALE PET','LIME CRUSH JUICE']
water = ['BOTTLED DRINKING WATER']
fruit_drinks = ['FIT O MIXED FRUIT','FIT O ORANGE','FIT O MANGO','TWISTEE APPLE']
milk_drinks = ['CHOCOLATE MILK','STRAWBERRY MILK']

In [36]:
store_table

,shop_id,shop_area_sq_ft,shop_profile,tot_sales,oct_sales,nov_1_sales,nov_2_sales,dec_sales,tot_customers,oct_customers,...,nov_2_customers,dec_customers,repeat_customers,frequent_customers,customers_180_bottles,customers_200_bottles,customers_400_bottles,customers_500_bottles,customers_1000_bottles,customers_1500_bottles
0,SHOP047,528,Moderate,838280,307505.0,147830.0,191790.0,191155,928,370.0,...,256.0,291,318,4.0,55,76,21.0,301,238,422
1,SHOP009,676,High,1966710,586745.0,290585.0,415425.0,673955,2498,898.0,...,716.0,971,902,4.0,85,151,37.0,1000,490,1196
2,SHOP083,676,Low,1691985,556160.0,279740.0,378780.0,477305,1900,697.0,...,545.0,691,700,9.0,84,112,17.0,596,503,956
3,SHOP117,676,Low,2325980,614355.0,403115.0,530415.0,778095,2037,739.0,...,601.0,807,841,9.0,73,138,20.0,696,635,1017
4,SHOP042,676,Low,1340215,390960.0,207075.0,321510.0,420670,1841,641.0,...,519.0,700,594,9.0,68,87,21.0,636,453,881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,SHOP087,527,NaN,1128935,297055.0,140540.0,360755.0,330585,1083,387.0,...,306.0,393,405,3.0,60,43,15.0,314,243,608
120,SHOP050,411,NaN,2399125,649065.0,248730.0,518975.0,982355,1778,730.0,...,573.0,835,822,13.0,57,93,23.0,548,342,1166
121,SHOP061,699,NaN,2106120,453755.0,209795.0,781090.0,661480,1745,609.0,...,507.0,658,641,3.0,87,125,20.0,387,503,991
122,SHOP056,597,NaN,3018540,872410.0,575745.0,585555.0,984830,2831,1012.0,...,865.0,1186,1125,22.0,119,236,30.0,919,634,1522


In [37]:
for drink in soft_drinks:
    drink_db = table.loc[table.beverage_description == drink]
    drink_db = pd.DataFrame(table.loc[drink_db.index].groupby(['shop_id'],sort=False)['amount'].sum(), index=all_shop_index)
    drink_db = drink_db.amount.fillna(0)
    
    drink_db = pd.Series(drink_db)
#     store_table = add_new_feature(store_table, drink, drink_db)
    store_table[drink] = 0
    i=0
    for idx in store_table.shop_id:
        store_table.loc[i, drink] = drink_db.loc[idx]
        i+=1

In [38]:
store_table

,shop_id,shop_area_sq_ft,shop_profile,tot_sales,oct_sales,nov_1_sales,nov_2_sales,dec_sales,tot_customers,oct_customers,...,CREAM SODA APPLE POP,SODA,LEMONADE,KIK COLA,SODA PET,NECTO,GINGER BEER SUGAR FREE,ORANGE CRUSH,DRY GINGER ALE PET,LIME CRUSH JUICE
0,SHOP047,528,Moderate,838280,307505.0,147830.0,191790.0,191155,928,370.0,...,11850,85990,15980,840,43910,34780,3280,37130,1760,0
1,SHOP009,676,High,1966710,586745.0,290585.0,415425.0,673955,2498,898.0,...,45150,252180,64480,5400,37040,122600,11120,62300,13220,0
2,SHOP083,676,Low,1691985,556160.0,279740.0,378780.0,477305,1900,697.0,...,36750,162610,64340,15840,74130,109720,7840,51610,23480,0
3,SHOP117,676,Low,2325980,614355.0,403115.0,530415.0,778095,2037,739.0,...,81750,233280,65180,10800,182620,180160,17600,47380,48450,7800
4,SHOP042,676,Low,1340215,390960.0,207075.0,321510.0,420670,1841,641.0,...,29400,152820,45420,3360,62120,54020,7280,30350,14860,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,SHOP087,527,NaN,1128935,297055.0,140540.0,360755.0,330585,1083,387.0,...,18900,61480,43680,2400,60160,54410,3840,28970,17650,0
120,SHOP050,411,NaN,2399125,649065.0,248730.0,518975.0,982355,1778,730.0,...,48000,154470,78440,4320,91990,178760,15760,40680,25920,6500
121,SHOP061,699,NaN,2106120,453755.0,209795.0,781090.0,661480,1745,609.0,...,39600,135840,61200,6240,133430,125870,7200,27100,34160,0
122,SHOP056,597,NaN,3018540,872410.0,575745.0,585555.0,984830,2831,1012.0,...,83550,416100,131100,8880,44320,195490,29360,58280,20250,6500


In [39]:
milk_sale_data = pd.Series(table['beverage_description'].isin(milk_drinks))
milk_sales = pd.DataFrame(table.loc[milk_sale_data].groupby('shop_id',sort=False)['amount'].sum(), index=all_shop_index)
store_table = add_new_feature(store_table, 'milk_sales', milk_sales)

In [40]:
soft_d_sale_data = pd.Series(table['beverage_description'].isin(soft_drinks))
soft_d_sales = pd.DataFrame(table.loc[soft_d_sale_data].groupby('shop_id',sort=False)['amount'].sum(), index=all_shop_index)
store_table = add_new_feature(store_table, 'soft_d_sales', soft_d_sales)

In [41]:
water_sale_data = pd.Series(table['beverage_description'].isin(water))
water_sales = pd.DataFrame(table.loc[water_sale_data].groupby('shop_id',sort=False)['amount'].sum(), index=all_shop_index)
store_table = add_new_feature(store_table, 'water_sales', water_sales)

In [42]:
fruit_d_sale_data = pd.Series(table['beverage_description'].isin(fruit_drinks))
fruit_d_sales = pd.DataFrame(table.loc[fruit_d_sale_data].groupby('shop_id',sort=False)['amount'].sum(), index=all_shop_index)
store_table = add_new_feature(store_table, 'fruit_d_sales', fruit_d_sales)

In [43]:
invoice_data = pd.DataFrame(table.groupby(['shop_id','invoice_id'])['amount'].sum(),)
avg_bill_value = pd.DataFrame(invoice_data.groupby('shop_id')['amount'].mean().round(2))
store_table = add_new_feature(store_table, 'avg_bill_value', avg_bill_value)

In [44]:
weekday_avg_sales = table.groupby(['weekday','shop_id'])['amount'].sum()
saturday_avg_sales = pd.DataFrame(weekday_avg_sales[5])
sunday_avg_sales = pd.DataFrame(weekday_avg_sales[6])
weekend_avg_sales = pd.DataFrame(round((saturday_avg_sales['amount'] + sunday_avg_sales['amount'])/18,2))
store_table = add_new_feature(store_table, 'weekend_avg_sales', weekend_avg_sales)

In [45]:
customer_per_day = pd.DataFrame(table.groupby(['shop_id','month','date'])['customer_id'].nunique())
customer_per_day = pd.DataFrame(customer_per_day.groupby('shop_id')['customer_id'].mean().round(0)).astype(int)
store_table = add_new_feature(store_table, 'customer_per_day', customer_per_day)

In [46]:
tot_orders = pd.DataFrame(table.groupby(['shop_id','month','date'])['invoice_id'].nunique())
tot_orders = pd.DataFrame(tot_orders.groupby('shop_id')['invoice_id'].sum().round(0)).astype(int)
store_table = add_new_feature(store_table,'tot_orders',tot_orders)

In [47]:
store_table.tail(24)

,shop_id,shop_area_sq_ft,shop_profile,tot_sales,oct_sales,nov_1_sales,nov_2_sales,dec_sales,tot_customers,oct_customers,...,DRY GINGER ALE PET,LIME CRUSH JUICE,milk_sales,soft_d_sales,water_sales,fruit_d_sales,avg_bill_value,weekend_avg_sales,customer_per_day,tot_orders
100,SHOP046,545,NaN,2607865,556485.0,291355.0,879035.0,880990,1736,679.0,...,28410,0,34640,1743320,578515,251390,944.54,32302.22,44,2761
101,SHOP024,676,NaN,1308795,335055.0,275830.0,325445.0,372465,1641,575.0,...,17370,0,22880,1075760,63525,146630,565.35,22019.72,37,2315
102,SHOP023,617,NaN,2553330,916485.0,445670.0,544750.0,646425,2227,799.0,...,24980,0,65750,1759360,185710,542510,802.68,50219.17,51,3181
103,SHOP097,310,NaN,2375735,610460.0,580165.0,477535.0,707575,1667,600.0,...,90670,0,221930,1660760,160895,332150,962.23,38967.78,40,2469
104,SHOP044,715,NaN,1251090,348525.0,184870.0,299685.0,418010,1550,546.0,...,34310,0,37190,1087740,9870,116290,598.89,28303.06,34,2089
105,SHOP030,530,NaN,2754185,1003750.0,343845.0,539280.0,867310,1636,696.0,...,27690,14300,75365,1963090,347690,368040,989.65,41864.72,44,2783
106,SHOP038,411,NaN,1444025,407920.0,207175.0,295035.0,533895,1781,681.0,...,12920,0,27950,1104080,98035,213960,553.48,30876.94,42,2609
107,SHOP029,597,NaN,1962105,489030.0,323595.0,534205.0,615275,2052,601.0,...,31190,0,55100,1363780,203245,339980,674.96,46140.28,47,2907
108,SHOP096,676,NaN,2600915,681745.0,372665.0,653965.0,892540,2497,911.0,...,29870,0,29175,2190780,105350,275610,692.84,48126.39,60,3754
109,SHOP092,617,NaN,1961240,504345.0,334230.0,634050.0,488615,2247,786.0,...,21910,0,51300,1375610,51870,482460,661.24,35130.83,48,2966


########## View the relationships between variables; color code by species type
di = {0: 'Low', 1.0: 'Moderate', 2.0:'High'} # dictionary

plt.figure()
sns.pairplot(store_table[:,fea_selected].replace({'shop_profile': di}), hue = "shop_profile", size=3, markers=["o", "s", "D"])
plt.show()

## Futher Feature Combining

To simplify and combine some of the features

Using Sales Growth to emphasize instead of separate monthly sale figures

In [48]:
store_table['oct_sales_ratio'] = (store_table['oct_sales'] / store_table['tot_sales'])
store_table['nov1_sales_ratio'] = (store_table['nov_1_sales'] / store_table['tot_sales'])
store_table['nov2_sales_ratio'] = (store_table['nov_2_sales'] / store_table['tot_sales'])
store_table['dec_sales_ratio'] = (store_table['dec_sales'] / store_table['tot_sales'])

store_table = store_table.drop(columns=['oct_sales','nov_1_sales','nov_2_sales','dec_sales'])

In [49]:
store_table['footprint_perday'] = store_table['customer_per_day'] / store_table['shop_area_sq_ft']

In [50]:
# store_table['customer_growth_vs_oct'] = (store_table['dec_customers'] / store_table['oct_customers']) + (store_table['nov_customers'] / store_table['oct_customers'])
# store_table = store_table.drop(columns=['oct_customers','nov_customers','dec_customers'])

In [51]:
store_table = store_table.loc[store_table.shop_id != "SHOP008"]
store_table

,shop_id,shop_area_sq_ft,shop_profile,tot_sales,tot_customers,oct_customers,nov_1_customers,nov_2_customers,dec_customers,repeat_customers,...,fruit_d_sales,avg_bill_value,weekend_avg_sales,customer_per_day,tot_orders,oct_sales_ratio,nov1_sales_ratio,nov2_sales_ratio,dec_sales_ratio,footprint_perday
0,SHOP047,528,Moderate,838280,928,370.0,219.0,256.0,291,318,...,120760,641.38,17221.94,21,1307,0.366829,0.176349,0.228790,0.228032,0.039773
1,SHOP009,676,High,1966710,2498,898.0,551.0,716.0,971,902,...,355420,555.57,36620.28,57,3540,0.298338,0.147752,0.211228,0.342681,0.084320
2,SHOP083,676,Low,1691985,1900,697.0,461.0,545.0,691,700,...,229080,617.51,34137.50,44,2740,0.328703,0.165332,0.223867,0.282098,0.065089
3,SHOP117,676,Low,2325980,2037,739.0,540.0,601.0,807,841,...,342370,730.75,50193.33,51,3183,0.264127,0.173310,0.228039,0.334524,0.075444
4,SHOP042,676,Low,1340215,1841,641.0,410.0,519.0,700,594,...,184920,530.99,29543.61,40,2524,0.291714,0.154509,0.239894,0.313882,0.059172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,SHOP087,527,NaN,1128935,1083,387.0,244.0,306.0,393,405,...,102210,742.72,18116.11,24,1520,0.263129,0.124489,0.319553,0.292829,0.045541
120,SHOP050,411,NaN,2399125,1778,730.0,389.0,573.0,835,822,...,345290,754.44,40565.28,51,3180,0.270542,0.103675,0.216318,0.409464,0.124088
121,SHOP061,699,NaN,2106120,1745,609.0,346.0,507.0,658,641,...,558160,886.41,60756.39,38,2376,0.215446,0.099612,0.370867,0.314075,0.054363
122,SHOP056,597,NaN,3018540,2831,1012.0,638.0,865.0,1186,1125,...,596900,686.03,58602.50,71,4400,0.289017,0.190736,0.193986,0.326260,0.118928


In [52]:
store_labelled = store_table.loc[:99]
store_labelled

,shop_id,shop_area_sq_ft,shop_profile,tot_sales,tot_customers,oct_customers,nov_1_customers,nov_2_customers,dec_customers,repeat_customers,...,fruit_d_sales,avg_bill_value,weekend_avg_sales,customer_per_day,tot_orders,oct_sales_ratio,nov1_sales_ratio,nov2_sales_ratio,dec_sales_ratio,footprint_perday
0,SHOP047,528,Moderate,838280,928,370.0,219.0,256.0,291,318,...,120760,641.38,17221.94,21,1307,0.366829,0.176349,0.228790,0.228032,0.039773
1,SHOP009,676,High,1966710,2498,898.0,551.0,716.0,971,902,...,355420,555.57,36620.28,57,3540,0.298338,0.147752,0.211228,0.342681,0.084320
2,SHOP083,676,Low,1691985,1900,697.0,461.0,545.0,691,700,...,229080,617.51,34137.50,44,2740,0.328703,0.165332,0.223867,0.282098,0.065089
3,SHOP117,676,Low,2325980,2037,739.0,540.0,601.0,807,841,...,342370,730.75,50193.33,51,3183,0.264127,0.173310,0.228039,0.334524,0.075444
4,SHOP042,676,Low,1340215,1841,641.0,410.0,519.0,700,594,...,184920,530.99,29543.61,40,2524,0.291714,0.154509,0.239894,0.313882,0.059172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,SHOP124,606,High,2967190,2281,817.0,547.0,685.0,857,877,...,433990,898.33,48450.28,53,3303,0.237991,0.135955,0.324607,0.301447,0.087459
96,SHOP012,336,Low,1102670,1262,387.0,284.0,381.0,527,408,...,123050,586.84,12960.56,30,1879,0.209736,0.150117,0.231629,0.408518,0.089286
97,SHOP093,705,High,1726515,1493,493.0,317.0,378.0,515,473,...,377770,922.28,33358.61,30,1872,0.475113,0.171122,0.136804,0.216960,0.042553
98,SHOP004,516,Low,1571700,1744,634.0,413.0,535.0,693,689,...,212990,590.64,33964.44,43,2661,0.259786,0.137027,0.236642,0.366546,0.083333


In [53]:
store_test = store_table.loc[100:]
store_test

,shop_id,shop_area_sq_ft,shop_profile,tot_sales,tot_customers,oct_customers,nov_1_customers,nov_2_customers,dec_customers,repeat_customers,...,fruit_d_sales,avg_bill_value,weekend_avg_sales,customer_per_day,tot_orders,oct_sales_ratio,nov1_sales_ratio,nov2_sales_ratio,dec_sales_ratio,footprint_perday
100,SHOP046,545,NaN,2607865,1736,679.0,420.0,525.0,639,683,...,251390,944.54,32302.22,44,2761,0.213387,0.111722,0.337071,0.337820,0.080734
101,SHOP024,676,NaN,1308795,1641,575.0,387.0,523.0,568,578,...,146630,565.35,22019.72,37,2315,0.256003,0.210751,0.248660,0.284586,0.054734
102,SHOP023,617,NaN,2553330,2227,799.0,530.0,665.0,792,792,...,542510,802.68,50219.17,51,3181,0.358937,0.174545,0.213349,0.253169,0.082658
103,SHOP097,310,NaN,2375735,1667,600.0,401.0,491.0,636,630,...,332150,962.23,38967.78,40,2469,0.256956,0.244204,0.201005,0.297834,0.129032
104,SHOP044,715,NaN,1251090,1550,546.0,332.0,453.0,517,510,...,116290,598.89,28303.06,34,2089,0.278577,0.147767,0.239539,0.334117,0.047552
105,SHOP030,530,NaN,2754185,1636,696.0,374.0,515.0,625,719,...,368040,989.65,41864.72,44,2783,0.364445,0.124845,0.195804,0.314906,0.083019
106,SHOP038,411,NaN,1444025,1781,681.0,405.0,518.0,665,634,...,213960,553.48,30876.94,42,2609,0.282488,0.143471,0.204314,0.369727,0.102190
107,SHOP029,597,NaN,1962105,2052,601.0,364.0,645.0,898,751,...,339980,674.96,46140.28,47,2907,0.249237,0.164922,0.272261,0.313579,0.078727
108,SHOP096,676,NaN,2600915,2497,911.0,524.0,737.0,1048,978,...,275610,692.84,48126.39,60,3754,0.262117,0.143282,0.251437,0.343164,0.088757
109,SHOP092,617,NaN,1961240,2247,786.0,532.0,590.0,767,745,...,482460,661.24,35130.83,48,2966,0.257156,0.170418,0.323290,0.249136,0.077796


## Split to train and validation

In [54]:
from sklearn.model_selection import train_test_split
store_train, store_val = train_test_split(store_labelled,test_size = 0.25, random_state = 1)

In [55]:
store_train_x = store_train.drop(columns=['shop_profile'])
store_train_y = store_train.shop_profile

store_val_x = store_val.drop(columns=['shop_profile'])
store_val_y = store_val.shop_profile

store_test_x = store_test.drop(columns=['shop_profile'])

In [56]:
store_train_x.index = store_train_x.shop_id
store_train_x = store_train_x.drop(columns=['shop_id'])

store_val_x.index = store_val_x.shop_id
store_val_x = store_val_x.drop(columns=['shop_id'])

store_test_x.index = store_test_x.shop_id
store_test_x = store_test_x.drop(columns=['shop_id'])

## Feature Scaling

In [57]:
std_scaler = StandardScaler()
data_cols = store_train_x.columns
train_x = std_scaler.fit_transform(store_train_x)
val_x = std_scaler.transform(store_val_x)
test_x = std_scaler.transform(store_test_x)

In [58]:
train_x = pd.DataFrame(train_x, columns=data_cols)
val_x = pd.DataFrame(val_x, columns=data_cols)
test_x = pd.DataFrame(test_x, columns=data_cols)

In [59]:
store_train_y = store_train_y.replace('Low', 0)
store_train_y = store_train_y.replace('Moderate', 1)
store_train_y = store_train_y.replace('High', 2)

store_val_y = store_val_y.replace('Low', 0)
store_val_y = store_val_y.replace('Moderate', 1)
store_val_y = store_val_y.replace('High', 2)

## Feature Selection

In [60]:
feature_sel_model = svm.SVC(kernel='linear')

In [61]:
recur_fe = RFE(estimator=feature_sel_model, n_features_to_select = 0.6)
features = recur_fe.fit(train_x, store_train_y, )

print("Number of features selected = " + str(features.n_features_))
bool_features_sel = features.support_
rank_features_sel = features.ranking_

Number of features selected = 25


In [62]:
col=0
list_features_sel = []
for i in bool_features_sel:
    if i:
        list_features_sel.append(train_x.columns[col])
    col+=1
list_features_sel

['shop_area_sq_ft',
 'oct_customers',
 'nov_2_customers',
 'dec_customers',
 'frequent_customers',
 'customers_180_bottles',
 'customers_500_bottles',
 'customers_1000_bottles',
 'TONIC PET',
 'CREAM SODA',
 'GINGER BEER',
 'CREAM SODA APPLE POP',
 'LEMONADE',
 'KIK COLA',
 'SODA PET',
 'NECTO',
 'GINGER BEER SUGAR FREE',
 'ORANGE CRUSH',
 'DRY GINGER ALE PET',
 'LIME CRUSH JUICE',
 'milk_sales',
 'water_sales',
 'fruit_d_sales',
 'avg_bill_value',
 'customer_per_day']

In [63]:
train_x_sel = train_x.loc[:,list_features_sel]
val_x_sel = val_x.loc[:,list_features_sel]
test_x_sel = test_x.loc[:,list_features_sel]

In [64]:
store_table.columns

Index(['shop_id', 'shop_area_sq_ft', 'shop_profile', 'tot_sales',
       'tot_customers', 'oct_customers', 'nov_1_customers', 'nov_2_customers',
       'dec_customers', 'repeat_customers', 'frequent_customers',
       'customers_180_bottles', 'customers_200_bottles',
       'customers_400_bottles', 'customers_500_bottles',
       'customers_1000_bottles', 'customers_1500_bottles', 'ORANGE BARLEY',
       'TONIC PET', 'CREAM SODA', 'GINGER BEER', 'CREAM SODA APPLE POP',
       'SODA', 'LEMONADE', 'KIK COLA', 'SODA PET', 'NECTO',
       'GINGER BEER SUGAR FREE', 'ORANGE CRUSH', 'DRY GINGER ALE PET',
       'LIME CRUSH JUICE', 'milk_sales', 'soft_d_sales', 'water_sales',
       'fruit_d_sales', 'avg_bill_value', 'weekend_avg_sales',
       'customer_per_day', 'tot_orders', 'oct_sales_ratio', 'nov1_sales_ratio',
       'nov2_sales_ratio', 'dec_sales_ratio', 'footprint_perday'],
      dtype='object')

## Model Training

In [65]:
model = svm.SVC(kernel='rbf')
# model = RandomForestRegressor()
# model = LinearRegression()

In [66]:
model.fit(train_x_sel, store_train_y)
train_pred_y_for = model.predict(train_x_sel).round()
print(f1_score(store_train_y,train_pred_y_for,average='micro'))
train_pred_y_val = model.predict(val_x_sel).round()
print(f1_score(store_val_y, train_pred_y_val,average='micro'))

0.8918918918918919
0.72


## Predicting

In [67]:
test_pred_y = model.predict(test_x_sel).round()
test_pred_y = pd.DataFrame(test_pred_y)
test_pred_y = test_pred_y.replace(0,'Low')
test_pred_y = test_pred_y.replace(1,'Moderate')
test_pred_y = test_pred_y.replace(2,'High')
test_pred_y.rename(columns={0:'shop_profile'},inplace=True)
test_pred_y

,shop_profile
0,High
1,Low
2,Low
3,High
4,Moderate
5,High
6,Low
7,Low
8,Low
9,Low


In [68]:
test_db = test_db.drop(columns=['shop_profile'])

In [69]:
submission = pd.merge(test_db, test_pred_y, left_index=True, right_index=True)
submission

,shop_id,shop_profile
0,SHOP046,High
1,SHOP024,Low
2,SHOP023,Low
3,SHOP097,High
4,SHOP044,Moderate
5,SHOP030,High
6,SHOP038,Low
7,SHOP029,Low
8,SHOP096,Low
9,SHOP092,Low


In [70]:
submission.shop_profile.value_counts()

Low         12
High        10
Moderate     2
Name: shop_profile, dtype: int64

In [71]:
submission.to_csv('submission2.csv', index=False)